In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

#import spacy
import numpy as np

import random
import math
import time


import pandas as pd
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
np.set_printoptions(threshold=3000)  
np.set_printoptions(linewidth=170)

#data_path = '/home/zhangxz/workspace/data/R1584_U50_V2/UIT.csv'
data_path = '/home/ubuntu/data/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv',header=None)
UIT

,0,1,2,3,4,5,6,7,8,9,10,11
0,365,3391,0,0,148,1030,101001,0,0,11807,1,2
1,203,5779,0,0,7,1030,10203,0,0,15068,1,2
2,208,4675,0,0,92,1035,10203,0,0,5375,1,2
3,159,332,0,0,56,1030,10202,0,0,5992,1,2
4,50,674,0,0,439,1030,10203,0,0,3468,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,29,2591880,34,1030,10203,0,0,10010,1,2
300979,158,8448,29,2591880,34,1030,10203,0,0,23340,1,2
300980,483,6463,29,2591940,35,1030,10203,0,0,10010,1,2
300981,158,4715,29,2591940,34,1030,10203,0,0,23340,1,2


In [3]:
level = 7


#UIT[4] = UIT[level]
#UIT=UIT[UIT[2]<18]
UIT[2] = UIT[3]//(60*60)
print(UIT.tail)
UITGroupTem = UIT.groupby(UIT[7])
TrainGroupIndex = set(UIT[7])
TrainGroup = []



for index in TrainGroupIndex:
    TrainGroup.append(UITGroupTem.get_group(index).reset_index(drop=True).drop([col for col in range(3,len(UIT.columns))],axis=1))
    print(len(set(TrainGroup[0][1])))

<bound method NDFrame.tail of          0     1    2        3    4     5       6   7   8      9   10  11
0       365  3391    0        0  148  1030  101001   0   0  11807   1   2
1       203  5779    0        0    7  1030   10203   0   0  15068   1   2
2       208  4675    0        0   92  1035   10203   0   0   5375   1   2
3       159   332    0        0   56  1030   10202   0   0   5992   1   2
4        50   674    0        0  439  1030   10203   0   0   3468   1   2
...     ...   ...  ...      ...  ...   ...     ...  ..  ..    ...  ..  ..
300978  483  6831  719  2591880   34  1030   10203   0   0  10010   1   2
300979  158  8448  719  2591880   34  1030   10203   0   0  23340   1   2
300980  483  6463  719  2591940   35  1030   10203   0   0  10010   1   2
300981  158  4715  719  2591940   34  1030   10203   0   0  23340   1   2
300982  483  2021  719  2591940   34  1030   10203   0   0  10010   1   2

[300983 rows x 12 columns]>
10000


In [5]:
T_all = max(UIT[2])+1
item_d = 10000
T_all

720

In [6]:
Frq_hour = np.zeros((len(TrainGroupIndex),T_all,item_d))

In [7]:
for index in TrainGroupIndex:
    for t in range(T_all):
        timeTrace = TrainGroup[index][TrainGroup[index][2]==t]
        counts =  timeTrace[1].value_counts().to_frame('count')
        for itemIndex,count in counts.iterrows():
            Frq_hour[index, t,itemIndex] = count['count']/timeTrace.shape[0]

In [8]:
trainX_list = []
trainY_list =[]
testX_list =[]
#testY_list=[]

In [9]:
m = 20
k = 26
trainTime = 24*12
testTime = 24*6
fold = 0

In [17]:
for index in TrainGroupIndex:
    trainX_list = []
    trainY_list =[]
    testX_list =[]
    testY_list=[]
    for t in range(trainTime-k-m):
        trainX_list.append(Frq_hour[index].T[:,t:t+m])
        trainY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    trainX = np.concatenate(np.array(trainX_list),axis = 0)
    trainY = np.concatenate(np.array(trainY_list),axis = 0)
    for t in range(trainTime-m,trainTime+testTime-m):
        testX_list.append(Frq_hour[index].T[:,t:t+m])
        #testY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    testX = np.concatenate(np.array(testX_list),axis = 0)
    #testY = np.concatenate(np.array(testY_list),axis = 0)
    np.savetxt("fold{}_city{}_trainX.csv".format(fold,index), trainX, delimiter=",")
    np.savetxt("fold{}_city{}_trainY.csv".format(fold,index), trainY, delimiter=",")
    np.savetxt("fold{}_city{}_testX.csv".format(fold,index), testX, delimiter=",")
    #np.savetxt("fold{}_city{}_testY.csv".format(fold,index), testY, delimiter=",")

In [18]:
m = 20
k = 26
trainTime = 24*18
testTime = 24*6
fold = 1

In [19]:
for index in TrainGroupIndex:
    trainX_list = []
    trainY_list =[]
    testX_list =[]
    testY_list=[]
    for t in range(trainTime-k-m):
        trainX_list.append(Frq_hour[index].T[:,t:t+m])
        trainY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    trainX = np.concatenate(np.array(trainX_list),axis = 0)
    trainY = np.concatenate(np.array(trainY_list),axis = 0)
    for t in range(trainTime-m,trainTime+testTime-m):
        testX_list.append(Frq_hour[index].T[:,t:t+m])
        #testY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    testX = np.concatenate(np.array(testX_list),axis = 0)
    #testY = np.concatenate(np.array(testY_list),axis = 0)
    np.savetxt("fold{}_city{}_trainX.csv".format(fold,index), trainX, delimiter=",")
    np.savetxt("fold{}_city{}_trainY.csv".format(fold,index), trainY, delimiter=",")
    np.savetxt("fold{}_city{}_testX.csv".format(fold,index), testX, delimiter=",")
    #np.savetxt("fold{}_city{}_testY.csv".format(fold,index), testY, delimiter=",")

In [9]:
m = 20
k = 26
trainTime = 24*24
testTime = 24*6
fold = 2

In [11]:
for index in TrainGroupIndex:
    trainX_list = []
    trainY_list =[]
    testX_list =[]
    testY_list=[]
    for t in range(trainTime-k-m):
        trainX_list.append(Frq_hour[index].T[:,t:t+m])
        trainY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    trainX = np.concatenate(np.array(trainX_list),axis = 0)
    trainY = np.concatenate(np.array(trainY_list),axis = 0)
    for t in range(trainTime-k-m,trainTime+testTime-k-m):
        testX_list.append(Frq_hour[index].T[:,t:t+m])
        testY_list.append(Frq_hour[index].T[:,t+m:t+m+k])
    testX = np.concatenate(np.array(testX_list),axis = 0)
    testY = np.concatenate(np.array(testY_list),axis = 0)
    np.savetxt("fold{}_city{}_trainX.csv".format(fold,index), trainX, delimiter=",")
    np.savetxt("fold{}_city{}_trainY.csv".format(fold,index), trainY, delimiter=",")
    np.savetxt("fold{}_city{}_testX.csv".format(fold,index), testX, delimiter=",")
    np.savetxt("fold{}_city{}_testY.csv".format(fold,index), testY, delimiter=",")

In [14]:
len(trainX_list)

530

In [216]:
np.savetxt("fold{}_city{}_trainX.csv".format(fold,index), trainX, delimiter=",")

In [196]:
trainX.append(Frq_hour[0].T[:,0:20])

In [192]:
trainX.shape

(20000, 20)

In [197]:
trainX.append(Frq_hour[0].T[:,1:21])

In [204]:
np.array(trainX).shape

(2, 20000, 20)

In [203]:
np.concatenate(np.array(trainX),axis = 0).shape

(40000, 20)